In [1]:
import os
import json
from random import sample
import cv2
import keras
import keras.backend as K
import numpy as np
import pandas as pd
import random as rn
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from tensorflow.image import grayscale_to_rgb
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split,KFold, cross_val_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from tensorflow.keras.metrics import mean_squared_error as mse
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D,BatchNormalization,Dropout,Conv2D,MaxPool2D

In [2]:
mse([[1,2],[1,2]],[[3,6],[3,6]])#, mse([1])
# mse([0],[3])

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_data():

    labels = []    
    imgs=[]
    path = '/content/drive/MyDrive/outputpairs/'
    for input_path in os.listdir(path):

        facehair_path = path+'/'+input_path + '/facehair.png'
        garment_top_path = path+'/'+input_path + '/garment_top.png'
        positions_path = path+'/'+input_path + '/positions.json'
        # facehair_img = cv2.imread(facehair_path,cv2.IMREAD_UNCHANGED)
        # facehair_img = cv2.cvtColor(facehair_img, cv2.COLOR_BGR2RGB)
        facehair_img = np.array(Image.open(facehair_path).convert('L'))
        garment_top_img = np.array(Image.open(garment_top_path).convert('L'))[:512,:512]

        shape = facehair_img.shape
        gap = max(shape)-min(shape) 
        facehair_img = cv2.resize(np.pad(facehair_img, [(0,),(gap//2,)]),(512,512))
        img = np.array([facehair_img, garment_top_img])
        img = np.einsum('ijk->jki',img)
        # garment_top_img = cv2.imread(garment_top_path,cv2.IMREAD_UNCHANGED)
        positions = json.load(open(positions_path))

        x = positions['x']
        y = positions['y']
        labels.append([x,y])
        imgs.append(img)
                                
    return np.array(imgs), np.array(labels)

In [ ]:
path = '/content/drive/MyDrive/outputpairs/'
out = '/content/drive/MyDrive/TRI/test/'

for input_path in sample(os.listdir(path),50):

    facehair_path = path+'/'+input_path + '/facehair.png'
    garment_top_path = path+'/'+input_path + '/garment_top.png'
    positions_path = path+'/'+input_path + '/positions.json'
    # facehair_img = cv2.imread(facehair_path,cv2.IMREAD_UNCHANGED)
    # facehair_img = cv2.cvtColor(facehair_img, cv2.COLOR_BGR2RGB)
    # face = Image.open(facehair_path)
    facehair_img = np.array(Image.open(facehair_path).convert('L'))

    # garment = Image.open(garment_top_path)
    garment_top_img = np.array(Image.open(garment_top_path).convert('L'))[:512,:512]

    face = cv2.imread(facehair_path,cv2.IMREAD_UNCHANGED)
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    garment = cv2.imread(garment_top_path,cv2.IMREAD_UNCHANGED)
    garment = cv2.cvtColor(garment, cv2.COLOR_BGR2RGB)
    
    shape = facehair_img.shape
    gap = max(shape)-min(shape) 
    facehair_img = cv2.resize(np.pad(facehair_img, [(0,),(gap//2,)]),(512,512))
    img = np.array([facehair_img, garment_top_img])
    img = np.array([np.einsum('ijk->jki',img)])
    positions = json.load(open(positions_path))

    [[x,y]] = model.predict(img)
    x,y = int(x),int(y)
    h,w,c = face.shape
    # print(h,w,c,x,y)
    output_img = garment
    output_img[y:y+h,x:x+w] = face
    plt.imsave(out + input_path +'.png',output_img)
    print(x,y,positions['x'],positions['y'])
    # plt.imshow(output_img)
    

1/1 [==============================] - 0s 24ms/step
326 0 324 33
1/1 [==============================] - 0s 25ms/step
283 0 264 10
1/1 [==============================] - 0s 22ms/step
294 0 319 10
1/1 [==============================] - 0s 21ms/step
271 0 319 74
1/1 [==============================] - 0s 21ms/step
303 0 291 55
1/1 [==============================] - 0s 22ms/step
250 0 249 15
1/1 [==============================] - 0s 24ms/step
271 0 273 10
1/1 [==============================] - 0s 27ms/step
304 0 311 14
1/1 [==============================] - 0s 21ms/step
275 0 295 12
1/1 [==============================] - 0s 25ms/step
348 0 336 11
1/1 [==============================] - 0s 21ms/step
259 0 266 13
1/1 [==============================] - 0s 22ms/step
282 0 297 20
1/1 [==============================] - 0s 22ms/step
258 0 251 12
1/1 [==============================] - 0s 22ms/step
260 0 259 17
1/1 [==============================] - 0s 22ms/step
371 0 386 14
1/1 [====================

In [ ]:
X, y = load_data()

In [ ]:
with open('/content/drive/MyDrive/TRI/data.npy', 'wb') as f:
    np.save(f, X)
    np.save(f, y)

In [4]:
with open('/content/drive/MyDrive/TRI/data.npy', 'rb') as f:
    X = np.load(f)
    y = np.load(f)

In [5]:
X.shape, y.shape

((3126, 512, 512, 2), (3126, 2))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
model = Sequential()
# model.add(ResNet50(include_top=False, weights=None, input_shape=(512,512,2),pooling='max'))
model.add(keras.layers.Input(shape=(512,512,2)))
model.add(Conv2D(3,(3,3),padding='same'))
model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
# model.layers[0].layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(2, activation='relu'))

# model.layers[0].trainable = False
model.summary()

94765736/94765736 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 512, 512, 3)       57        
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                           

In [ ]:
optimizer = Adam(lr=0.00002)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mse'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# earlystopping = EarlyStopping(monitor='loss', verbose=1, patience=10, min_delta=0.02)
checkpoint = ModelCheckpoint('/content/drive/MyDrive/TRI/face3.h5', monitor='val_loss', mode='min', initial_value_threshold=800)

In [ ]:
model.load_weights('/content/drive/MyDrive/TRI/face2.h5')

In [ ]:
#batch_size = 32
history = model.fit(X_train,y_train,batch_size=16,validation_data=(X_test,y_test),
                              epochs=10, callbacks=[checkpoint])

Epoch 1/10
176/176 [==============================] - 156s 858ms/step - loss: 1116.7073 - mse: 1116.7073 - val_loss: 941.3594 - val_mse: 941.3594
Epoch 2/10
176/176 [==============================] - 150s 854ms/step - loss: 1070.7913 - mse: 1070.7913 - val_loss: 1051.5486 - val_mse: 1051.5486
Epoch 3/10
176/176 [==============================] - 151s 856ms/step - loss: 1063.5145 - mse: 1063.5145 - val_loss: 1071.8879 - val_mse: 1071.8879
Epoch 4/10
176/176 [==============================] - 152s 866ms/step - loss: 1064.4149 - mse: 1064.4149 - val_loss: 1102.7725 - val_mse: 1102.7725
Epoch 5/10
176/176 [==============================] - 151s 860ms/step - loss: 1060.3390 - mse: 1060.3390 - val_loss: 943.8538 - val_mse: 943.8538
Epoch 6/10
176/176 [==============================] - 151s 857ms/step - loss: 1082.6096 - mse: 1082.6096 - val_loss: 972.6164 - val_mse: 972.6164
Epoch 7/10
176/176 [==============================] - 151s 856ms/step - loss: 1067.1414 - mse: 1067.1414 - val_loss: 1

In [ ]:
#batch_size = 32
history = model.fit(X_train,y_train,batch_size=16,validation_data=(X_test,y_test),
                              epochs=5, callbacks=[checkpoint])

Epoch 1/5
176/176 [==============================] - 167s 879ms/step - loss: 1055.0948 - mse: 1055.0948 - val_loss: 848.3727 - val_mse: 848.3727
Epoch 2/5
176/176 [==============================] - 149s 848ms/step - loss: 1008.1448 - mse: 1008.1448 - val_loss: 871.4346 - val_mse: 871.4346
Epoch 3/5
176/176 [==============================] - 149s 847ms/step - loss: 1026.8696 - mse: 1026.8696 - val_loss: 856.0651 - val_mse: 856.0651
Epoch 4/5
176/176 [==============================] - 150s 852ms/step - loss: 993.6096 - mse: 993.6096 - val_loss: 843.6117 - val_mse: 843.6117
Epoch 5/5
176/176 [==============================] - 149s 848ms/step - loss: 996.3391 - mse: 996.3391 - val_loss: 842.8549 - val_mse: 842.8549


In [ ]:
K.set_value(model.optimizer.learning_rate, 0.0001)

In [ ]:
for layer in model.layers[1].layers[-70:]:
    layer.trainable = True

## Using MAPE and layer freezing

In [25]:
model = Sequential()
# model.add(ResNet50(include_top=False, weights=None, input_shape=(512,512,2),pooling='max'))
model.add(keras.layers.Input(shape=(512,512,2)))
model.add(Conv2D(3,(3,3),padding='same'))
model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
# model.layers[0].layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(2, activation='linear'))

# model.layers[0].trainable = False
# model.summary()


In [ ]:
model.layers[1].trainable = True

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 512, 512, 3)       57        
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_3 (Dense)             (None, 1024)              2098176   
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 batch_normalization_2 (Batc  (None, 1024)             4096      
 hNormalization)                                                 
                                                      

In [18]:
optimizer = Adam(lr=0.0005)
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mae'])

In [15]:
# earlystopping = EarlyStopping(monitor='loss', verbose=1, patience=10, min_delta=0.02)
checkpoint = ModelCheckpoint('/content/drive/MyDrive/TRI/face6.h5', monitor='val_loss', mode='min', initial_value_threshold=156)

In [22]:
model.load_weights('/content/drive/MyDrive/TRI/face5.h5')

In [ ]:
model.save_weights('/content/drive/MyDrive/TRI/face6.h5')

In [12]:
#batch_size = 32
history = model.fit(X_train,y_train,batch_size=16,validation_data=(X_test,y_test),
                              epochs=10, callbacks=[checkpoint])

Epoch 1/10
176/176 [==============================] - 173s 876ms/step - loss: 156.9214 - mae: 156.9214 - val_loss: 154.6269 - val_mae: 154.6269
Epoch 2/10
176/176 [==============================] - 146s 831ms/step - loss: 153.0546 - mae: 153.0546 - val_loss: 148.3881 - val_mae: 148.3881
Epoch 3/10
176/176 [==============================] - 146s 830ms/step - loss: 145.8095 - mae: 145.8095 - val_loss: 143.4685 - val_mae: 143.4685
Epoch 4/10
176/176 [==============================] - 146s 830ms/step - loss: 137.4012 - mae: 137.4012 - val_loss: 133.7784 - val_mae: 133.7784
Epoch 5/10
176/176 [==============================] - 146s 828ms/step - loss: 128.5354 - mae: 128.5354 - val_loss: 120.8517 - val_mae: 120.8517
Epoch 6/10
176/176 [==============================] - 146s 831ms/step - loss: 117.8687 - mae: 117.8687 - val_loss: 112.5677 - val_mae: 112.5677
Epoch 7/10
176/176 [==============================] - 146s 830ms/step - loss: 104.9000 - mae: 104.9000 - val_loss: 96.5987 - val_mae: 96

In [19]:
history = model.fit(X_train,y_train,batch_size=16,validation_data=(X_test,y_test),
                              epochs=5, callbacks=[checkpoint])

Epoch 1/5
176/176 [==============================] - 149s 818ms/step - loss: 40.8800 - mae: 40.8800 - val_loss: 29.8984 - val_mae: 29.8984
Epoch 2/5
176/176 [==============================] - 147s 834ms/step - loss: 31.8392 - mae: 31.8392 - val_loss: 24.6674 - val_mae: 24.6674
Epoch 3/5
176/176 [==============================] - 147s 835ms/step - loss: 27.8126 - mae: 27.8126 - val_loss: 32.7721 - val_mae: 32.7721
Epoch 4/5
176/176 [==============================] - 145s 826ms/step - loss: 26.7258 - mae: 26.7258 - val_loss: 25.3033 - val_mae: 25.3033
Epoch 5/5
176/176 [==============================] - 145s 826ms/step - loss: 26.7652 - mae: 26.7652 - val_loss: 24.6036 - val_mae: 24.6036


In [ ]:
history = model.fit(X_train,y_train,batch_size=16,validation_data=(X_test,y_test),
                              epochs=5, callbacks=[checkpoint])

Epoch 1/5
176/176 [==============================] - 174s 884ms/step - loss: 27.4739 - mae: 27.4739 - val_loss: 24.7931 - val_mae: 24.7931
Epoch 2/5
176/176 [==============================] - 150s 850ms/step - loss: 27.4106 - mae: 27.4106 - val_loss: 24.7793 - val_mae: 24.7793
Epoch 3/5
176/176 [==============================] - 148s 841ms/step - loss: 27.1157 - mae: 27.1157 - val_loss: 24.9379 - val_mae: 24.9379
Epoch 4/5
176/176 [==============================] - 148s 841ms/step - loss: 27.3561 - mae: 27.3561 - val_loss: 24.7209 - val_mae: 24.7209
Epoch 5/5
176/176 [==============================] - 148s 839ms/step - loss: 27.3170 - mae: 27.3170 - val_loss: 24.4947 - val_mae: 24.4947


In [ ]:
path = '/content/drive/MyDrive/outputpairs/'
out = '/content/drive/MyDrive/TRI/test2/'

for input_path in sample(os.listdir(path),20):

    facehair_path = path+'/'+input_path + '/facehair.png'
    garment_top_path = path+'/'+input_path + '/garment_top.png'
    positions_path = path+'/'+input_path + '/positions.json'
    # facehair_img = cv2.imread(facehair_path,cv2.IMREAD_UNCHANGED)
    # facehair_img = cv2.cvtColor(facehair_img, cv2.COLOR_BGR2RGB)
    # face = Image.open(facehair_path)
    facehair_img = np.array(Image.open(facehair_path).convert('L'))

    # garment = Image.open(garment_top_path)
    garment_top_img = np.array(Image.open(garment_top_path).convert('L'))[:512,:512]

    face = cv2.imread(facehair_path,cv2.IMREAD_UNCHANGED)
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    garment = cv2.imread(garment_top_path,cv2.IMREAD_UNCHANGED)
    garment = cv2.cvtColor(garment, cv2.COLOR_BGR2RGB)
    
    shape = facehair_img.shape
    gap = max(shape)-min(shape) 
    facehair_img = cv2.resize(np.pad(facehair_img, [(0,),(gap//2,)]),(512,512))
    img = np.array([facehair_img, garment_top_img])
    img = np.array([np.einsum('ijk->jki',img)])
    positions = json.load(open(positions_path))

    [[x,y]] = model.predict(img)
    x,y = int(x),int(y)
    h,w,c = face.shape
    # print(h,w,c,x,y)
    output_img = garment
    output_img[y:y+h,x:x+w] = face
    plt.imsave(out + input_path +'.png',output_img)
    print(x,y,positions['x'],positions['y'])
    # plt.imshow(output_img)
    